Hello Bridget!

I’m happy to review your project today.
I will mark your mistakes and give you some hints how it is possible to fix them. We are getting ready for real job, where your team leader/senior colleague will do exactly the same. Don't worry and study with pleasure!

Below you will find my comments - **please do not move, modify or delete them**.

You can find my comments in green, yellow or red boxes like this:

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Success. Everything is done succesfully.
</div>

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Remarks. Some recommendations.
</div>

<div class="alert alert-block alert-danger">

<b>Reviewer's comment</b> <a class="tocSkip"></a>

Needs fixing. The block requires some corrections. Work can't be accepted with the red comments.
</div>

You can answer me by using this:

<div class="alert alert-block alert-info">
<b>Student answer.</b> <a class="tocSkip"></a>

Text here.
</div>

Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

## Data preparation

### Import Libraries

In [3]:
import numpy as np
import pandas as pd
import math
import time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.linear_model
import sklearn.metrics
import sklearn.preprocessing
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from IPython.display import display
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

### Import data

In [4]:
df = pd.read_csv('/datasets/car_data.csv')

### Data Review

In [5]:
df.sample(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
334901,29/03/2016 03:36,1300,NaN,1910,NaN,0,NaN,5000,9,NaN,audi,NaN,29/03/2016 00:00,0,82256,05/04/2016 17:25
87135,18/03/2016 23:41,3799,small,2005,manual,199,punto,150000,10,gasoline,fiat,no,18/03/2016 00:00,0,74855,05/04/2016 23:16
97546,02/04/2016 15:39,500,sedan,1999,manual,101,golf,150000,5,petrol,volkswagen,no,02/04/2016 00:00,0,96524,02/04/2016 15:39
250315,24/03/2016 17:36,950,sedan,1998,auto,224,e_klasse,150000,11,petrol,mercedes_benz,NaN,24/03/2016 00:00,0,57462,03/04/2016 09:45
81231,11/03/2016 20:37,12250,wagon,2008,manual,177,3er,100000,11,gasoline,bmw,no,11/03/2016 00:00,0,3051,06/04/2016 02:46
307868,30/03/2016 12:57,2250,NaN,2017,manual,101,zafira,150000,11,gasoline,opel,no,30/03/2016 00:00,0,47137,07/04/2016 04:15
232852,20/03/2016 14:42,4000,sedan,2003,auto,177,e_klasse,150000,5,gasoline,mercedes_benz,no,20/03/2016 00:00,0,94060,20/03/2016 14:42
128809,29/03/2016 09:52,7100,suv,2004,auto,450,cayenne,150000,11,petrol,porsche,no,29/03/2016 00:00,0,42109,29/03/2016 09:52
270437,02/04/2016 13:48,2650,small,2005,manual,101,polo,150000,10,gasoline,volkswagen,no,02/04/2016 00:00,0,4860,06/04/2016 12:16
4046,04/04/2016 08:56,1990,wagon,1996,manual,110,NaN,150000,5,gasoline,audi,no,04/04/2016 00:00,0,15848,06/04/2016 10:16


In [6]:
#Lowercase columns
df.columns = df.columns.str.lower()

In [7]:
#Rename columns w/ underscores
df = df.rename(columns={
    'datecrawled': 'date_crawled',
    'vehicletype': 'vehicle_type',
    'registrationyear': 'registration_year',
    'registrationmonth': 'registration_month',
    'fueltype': 'fuel_type',
    'notrepaired': 'not_repaired',
    'datecreated': 'date_created',
    'numberofpictures': 'num_of_pictures',
    'postalcode': 'postal_code',
    'lastseen': 'last_seen'
})

In [8]:
print(df.shape)
df.info()

(354369, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   mileage             354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  num_of_pictures     354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  o

In [9]:
df.describe()

,price,registration_year,power,mileage,registration_month,num_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


**Notes:**

1. Convert to datetime
 -- date_crawled
 -- date_created
 -- last_seen
2. Check for missing values
3. Check for duplicates
4. Outliers
 -- registration_year (min of 1000 and max 9999)
 -- power (min of 0)
5. Drop last_seen (irrelevant) and Number of pictures (no data; all 0s)


<div class="alert alert-block alert-success">
<b>Reviewer's comment V1</b> <a class="tocSkip"></a>

Correct
    
</div>

### Data Preprocessing

In [10]:
#Convert columns to datetime
df[['date_crawled', 'date_created', 'last_seen']] = df[['date_crawled', 'date_created', 'last_seen']].apply(pd.to_datetime)

In [11]:
#Check for duplicated values
df.duplicated().sum()

262

In [12]:
#Drop duplicates
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
#Check for missing values
df.isna().sum()

date_crawled              0
price                     0
vehicle_type          37484
registration_year         0
gearbox               19830
power                     0
model                 19701
mileage                   0
registration_month        0
fuel_type             32889
brand                     0
not_repaired          71145
date_created              0
num_of_pictures           0
postal_code               0
last_seen                 0
dtype: int64

In [14]:
#Drop missing values because using One Hot Encoding instead of Label Encoder
df.dropna(inplace=True)

<div class="alert alert-block alert-warning">
<b>Reviewer's comment V1</b> <a class="tocSkip"></a>

Actually no one model including CatBoost can handle missing values. You handled NaNs with LabelEncoder. It encoded NaNs as additional category. You can check the data after encoding to see that there are not NaNs. So, you code didn't throw an error because of LabelEncoder and not because of CatBoost:)
    
</div>

<div class="alert alert-block alert-info">
<b>Student answer.</b> <a class="tocSkip"></a>
OK thank you for this info. Think I understand. I dropped missing values now since I didn't use LabelEncoder this time.
</div>

<div class="alert alert-block alert-warning">
<b>Reviewer's comment V2</b> <a class="tocSkip"></a>

When you work with ML models it's always not a good idea to remove a row because of NaN in one column because you lose information from other columns which can be useful for model training. So, instead of dropping NaNs it's better to fill them.
    
Previously everything was fine due to LabelEncoder. If you don't use LabelEncoder now, you need to fill all the missing values. One of the best way to fill NaNs in categorical columns is to fill them with a placeholder like string 'unknown'.
  
</div>

In [15]:
#Check unique value counts for missing data
df['registration_month'].value_counts()

3     26502
6     23354
4     21927
5     21562
7     20236
10    19663
9     18152
12    18094
11    18053
1     17378
8     16890
2     16225
0      7531
Name: registration_month, dtype: int64

In [16]:
#Impute missing values with most frequent month
most_frequent_month = df['registration_month'].mode()[0]
df['registration_month'] = df['registration_month'].replace(0, most_frequent_month)

In [17]:
#Handle outliers for registration_year
df = df[(df['registration_year'] >= 1920) & (df['registration_year'] <= 2025)]

In [18]:
#Handle outliers for power
df = df[df['power'] >= 100]

In [19]:
#Feature engineer a new time column from the relavant datetime ones
df['created_to_crawled'] =  df['date_crawled'] - df['date_created']

In [20]:
#Convert new feature to datetime with seconds as unit
df['created_to_crawled'] = df['created_to_crawled'].astype('int64') // 10**9 

In [21]:
#Drop last_seen and num_of_pictures columns because irrelavant; drop date columns now that new feature exists
df.drop(columns=['num_of_pictures', 'last_seen', 'date_crawled', 'date_created'], inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156209 entries, 5 to 354105
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               156209 non-null  int64 
 1   vehicle_type        156209 non-null  object
 2   registration_year   156209 non-null  int64 
 3   gearbox             156209 non-null  object
 4   power               156209 non-null  int64 
 5   model               156209 non-null  object
 6   mileage             156209 non-null  int64 
 7   registration_month  156209 non-null  int64 
 8   fuel_type           156209 non-null  object
 9   brand               156209 non-null  object
 10  not_repaired        156209 non-null  object
 11  postal_code         156209 non-null  int64 
 12  created_to_crawled  156209 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 16.7+ MB


In [23]:
#Rename the categorical features columns names to be used in LGB encoding
df.rename(columns={'vehicle_type': 'name_vehicle_type', 'gearbox': 'name_gearbox', 'model': 'name_model',
                   'fuel_type': 'name_fuel_type', 'brand': 'name_brand', 'not_repaired': 'name_not_repaired'
                  }, inplace=True)

In [24]:
#Convert columns to be used in categorical feature encoding process to category datatype
df[[
    'name_vehicle_type',
    'name_gearbox',
    'name_model',
    'name_fuel_type',
    'name_brand',
    'name_not_repaired'
]] = df[[
    'name_vehicle_type',
    'name_gearbox',
    'name_model',
    'name_fuel_type',
    'name_brand',
    'name_not_repaired'
]].astype('category')


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156209 entries, 5 to 354105
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   price               156209 non-null  int64   
 1   name_vehicle_type   156209 non-null  category
 2   registration_year   156209 non-null  int64   
 3   name_gearbox        156209 non-null  category
 4   power               156209 non-null  int64   
 5   name_model          156209 non-null  category
 6   mileage             156209 non-null  int64   
 7   registration_month  156209 non-null  int64   
 8   name_fuel_type      156209 non-null  category
 9   name_brand          156209 non-null  category
 10  name_not_repaired   156209 non-null  category
 11  postal_code         156209 non-null  int64   
 12  created_to_crawled  156209 non-null  int64   
dtypes: category(6), int64(7)
memory usage: 10.6 MB


In [26]:
#Scale numerical features before one hot encoding

numerical_features = ['registration_year', 'power', 'mileage', 'registration_month', 
                      'postal_code', 'created_to_crawled']

# Apply StandardScaler to numerical features
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

In [27]:
#Create categorical features variable to use for further processing
categorical_features = [
    'name_vehicle_type',
    'name_gearbox',
    'name_model',
    'name_fuel_type',
    'name_brand',
    'name_not_repaired'
]

In [28]:
#Review unique values for each feature
for feature in categorical_features:
    print(feature)
    print(df[feature].value_counts())
    print("-----------------------")

name_vehicle_type
sedan          55072
wagon          43824
bus            17938
convertible    12533
coupe          10641
suv             8471
small           6881
other            849
Name: name_vehicle_type, dtype: int64
-----------------------
name_gearbox
manual    113245
auto       42964
Name: name_gearbox, dtype: int64
-----------------------
name_model
3er           14288
other         11818
golf          11310
a4             7297
passat         6739
              ...  
sandero           1
rangerover        1
ypsilon           1
picanto           1
kalos             1
Name: name_model, Length: 234, dtype: int64
-----------------------
name_fuel_type
petrol      88249
gasoline    64499
lpg          3146
cng           224
hybrid         54
other          25
electric       12
Name: name_fuel_type, dtype: int64
-----------------------
name_brand
volkswagen       28815
bmw              26307
mercedes_benz    20399
audi             19827
opel             13231
ford              9164


In [29]:
for feature in categorical_features:
    print(feature, ":", df[feature].value_counts().count())
    print("-------------------------")

name_vehicle_type : 8
-------------------------
name_gearbox : 2
-------------------------
name_model : 234
-------------------------
name_fuel_type : 7
-------------------------
name_brand : 38
-------------------------
name_not_repaired : 2
-------------------------


In [30]:
#One Hot Encoding preperation
#Removing infrequent categories

threshold = 10

for column in categorical_features:
    value_counts = df[column].value_counts()
    to_remove = value_counts[value_counts < threshold].index
    df[column] = df[column].apply(lambda x: 'Other' if x in to_remove else x)

In [31]:
#Apply One-Hot Encoding to columns with fewer unique values
df_encoded = pd.get_dummies(df, columns=['name_vehicle_type', 'name_gearbox', 'name_fuel_type', 'name_not_repaired'], drop_first=True)

In [32]:
#Linear Regression data preparation
#Drop brand and model columns because there are too many unique values for one hot encoding

df_encoded.drop(columns=['name_model', 'name_brand'], inplace=True)

In [33]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156209 entries, 5 to 354105
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   price                          156209 non-null  int64  
 1   registration_year              156209 non-null  float64
 2   power                          156209 non-null  float64
 3   mileage                        156209 non-null  float64
 4   registration_month             156209 non-null  float64
 5   postal_code                    156209 non-null  float64
 6   created_to_crawled             156209 non-null  float64
 7   name_vehicle_type_convertible  156209 non-null  uint8  
 8   name_vehicle_type_coupe        156209 non-null  uint8  
 9   name_vehicle_type_other        156209 non-null  uint8  
 10  name_vehicle_type_sedan        156209 non-null  uint8  
 11  name_vehicle_type_small        156209 non-null  uint8  
 12  name_vehicle_type_suv         

<div class="alert alert-block alert-success">
<b>Reviewer's comment V1</b> <a class="tocSkip"></a>

Everythings is correct. Good job!
    
</div>

### Splitting X and y sets; Training, Test, and Validation sets

In [34]:
#Create X and y sets
#Original dataframe

X_original = df.drop(['price'], axis=1)
y_original = df['price']

#df_encoded dataframe

X_encoded = df_encoded.drop(['price'], axis=1)
y_encoded = df_encoded['price']

In [35]:
#Create train, test, and validation sets
#Original sets

X_original_temp, X_original_test, y_original_temp, y_original_test = train_test_split(
    X_original, y_original, test_size=0.2, random_state=101)

X_original_train, X_original_val, y_original_train, y_original_val = train_test_split(
    X_original_temp, y_original_temp, test_size=0.25, random_state=101)

#One Hot Encoded sets

X_encoded_temp, X_encoded_test, y_encoded_temp, y_encoded_test = train_test_split(
    X_encoded, y_encoded, test_size=0.2, random_state=101)

X_encoded_train, X_encoded_val, y_encoded_train, y_encoded_val = train_test_split(
    X_encoded_temp, y_encoded_temp, test_size=0.25, random_state=101)

In [36]:
print(X_encoded_train.shape)
print(y_encoded_train.shape)
print(X_original_train.shape)
print(y_original_train.shape)

(93725, 21)
(93725,)
(93725, 12)
(93725,)


**Conclusion:**

Performed the following data preprocessing:
1. Datetime conversion
2. Feature engineered new elapsed time between date features
3. Dropped duplicates
4. Addressed Outliers
5. Handled 0 values in month data
6. Dropped irrelevant columns
7. Created train, test, validation sets for original df and another set that's label encoded

## Model training

In [37]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

In [43]:
#Linear Regression as base model

linear_model = LinearRegression()
linear_model.fit(X_encoded_train, y_encoded_train)
y_encoded_val_pred = linear_model.predict(X_encoded_val)

#Get baseline RMSE
linear_rmse = mean_squared_error(y_encoded_val, y_encoded_val_pred, squared=False)

#Get baseline test RMSE
test_rmse_linear = np.sqrt(mean_squared_error(y_encoded_test, y_encoded_val_pred))

print(f'Baseline Linear Regression RMSE: {linear_rmse}')
print(f'Baseline Test Linear Regression RMSE: {test_rmse_linear}')

Baseline Linear Regression RMSE: 3370.619291921021
Baseline Test Linear Regression RMSE: 6281.211347513317


<div class="alert alert-block alert-danger">
<b>Reviewer's comment V1</b> <a class="tocSkip"></a>

1. LabelEncoder is not suitable for linear models. For linear models only one hot encoding can be used
2. For linear models all numerical features should be scaled. Not all features but only initially numerical features.
    
</div>

<div class="alert alert-block alert-info">
<b>Student answer.</b> <a class="tocSkip"></a>

Thanks for the feedback. This makes sense. I implemented both feedback suggestions above and reran linear regression and updated all conclusions accordingly
</div>

<div class="alert alert-block alert-warning">
<b>Reviewer's comment V2</b> <a class="tocSkip"></a>

Good job! But:
1. It's proper to use scaling after splitting the data because scaler should be trained only on train data as any other algorithm from sklearn.
2. Linear models can easily handle thousands of features. So, it's not necessary to remove features with a lot of unique values from ohe if you work with linear models only.
    
</div>

In [46]:
#Decision Tree

#Define the parameter grid
param_grid_dtree = {
    "criterion": ["mse"],
    "min_samples_split": [10, 20],
    "max_depth": [2, 6, 8],
    "min_samples_leaf": [20, 40, 100],
    "max_leaf_nodes": [5, 20, 100],
}

#Create the DecisionTreeRegressor model
dtree_model = DecisionTreeRegressor(random_state=101)

#Define GridSearchCV with the custom scorer
dtree_grid = GridSearchCV(dtree_model, param_grid_dtree, scoring=rmse_scorer, cv=KFold(n_splits=5, shuffle=True, random_state=101), verbose=2)

#Perform grid search
dtree_grid.fit(X_encoded_train, y_encoded_train)

#Get the best fitted model and measure training time
best_dtree_model = dtree_grid.best_estimator_
start_training_time_dtree = time.time()
best_dtree_model.fit(X_encoded_train, y_encoded_train)
training_time_dtree = time.time() - start_training_time_dtree

# Measure training time
start_training_time_dtree = time.time()
best_dtree_model = dtree_grid.best_estimator_
best_dtree_model.fit(X_encoded_train, y_encoded_train)
training_time_dtree = time.time() - start_training_time_dtree

#Measure prediction time
start_prediction_time_dtree = time.time()
predictions_dtree = best_dtree_model.predict(X_encoded_test)
prediction_time_dtree = time.time() - start_prediction_time_dtree

#Evaluate the model on the test set
test_rmse_dtree = np.sqrt(mean_squared_error(y_encoded_test, predictions_dtree))

#Perform cross-validation with the best estimator and custom scorer
scores_dtree = cross_val_score(best_dtree_model, X_encoded_train, y_encoded_train, cv=5, scoring=rmse_scorer)

#Print the cross-validation scores
print("RMSE:", np.abs(scores_dtree.mean()))
print("Test RMSE:", test_rmse_dtree)

# Print the training results
print("Training time:", training_time_dtree)
print("Prediction time:", prediction_time_dtree)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=20; total time=   0.1s
[C

[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_nodes=100, min_samples_leaf=40, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=2, max_leaf_no

[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=20; total time=   0.2s
[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=6, max_leaf_nodes=20, min_samples_leaf=100, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=6, max_leaf_no

[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=10; total time=   0.1s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=20; total time=   0.2s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, min_samples_leaf=20, min_samples_split=20; total time=   0.1s
[CV] END criterion=mse, max_depth=8, max_leaf_nodes=20, 

In [45]:
#Random Forest

#Define paramater grid
param_grid_rforest = {
    'n_estimators': [5, 10],
    'max_depth': [3, 6, 8],
    'min_samples_split': [2, 3, 6],
    'min_samples_leaf': [2, 3, 6]
}

#Create the model
rforest_model = RandomForestRegressor(random_state=101)

#Define GridSearchCV with the custom scorer
rforest_grid = GridSearchCV(rforest_model, param_grid = param_grid_rforest, scoring = rmse_scorer, 
                                  cv = KFold(n_splits = 5, shuffle = True, random_state = 101), verbose = 2)

#Perform grid search
rforest_grid.fit(X_encoded_train, y_encoded_train)


# Get the best fitted model and measure training time
best_rforest_model = rforest_grid.best_estimator_
start_training_time_rforest = time.time()
best_rforest_model.fit(X_encoded_train, y_encoded_train)
training_time_rforest = time.time() - start_training_time_rforest

#Measure prediction time
start_prediction_time_rforest = time.time()
predictions_rforest = best_rforest_model.predict(X_encoded_test)
prediction_time_rforest = time.time() - start_prediction_time_rforest

#Evaluate the model on the test set
test_rmse_rforest = np.sqrt(mean_squared_error(y_encoded_test, predictions_rforest))

#Perform cross-validation with the best estimator and custom scorer
scores_rforest = cross_val_score(best_rforest_model, X_encoded_train, y_encoded_train, 
                            cv=5, scoring=rmse_scorer)

#Print the cross-validation scores
print("RMSE:", np.abs(scores_rforest.mean()))
print("Test RMSE:", test_rmse_rforest)

# Print the training results
print("Training time:", training_time_rforest)
print("Prediction time:", prediction_time_rforest)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=2, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=2, min_sample

[CV] END max_depth=3, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=6, min_samples_split=6, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=6, min_samples_split=6, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=6, min_samples_split=6, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=6, min_samples_split=6, n_estimators=10; total time=   0.5s
[CV] END max_depth=3, min_samples_leaf=6, min_samples_split=6, n_estimators=10; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=2, min_samples_split=2, n_estimators=5; total time=   0.5s
[CV] END max_de

[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   0.9s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   1.0s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   1.0s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.5s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=6, n_estimators=10; total time=   1.0s
[CV] END max_depth=6, min_samples_leaf=6, min_samples_split=6, n_estimators=10; total time=   1.0s
[CV] END max_de

[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=5; total time=   0.7s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=5; total time=   0.6s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=5; total time=   0.6s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=5; total time=   0.7s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   1.3s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   1.2s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   1.2s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   1.2s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=3, n_estimators=10; total time=   1.2s
[CV] END max_depth=8, min_samples_leaf=6, min_samples_split=6, n_estimators=5; total time=   0.6s
[CV] END max_de

In [40]:
#LightGBM

#Define the parameter grid
param_grid_lgbm = {'n_estimators': [50, 100],
            'max_depth' : [1, 3, 12],
            'num_leaves': [31, 63],
            'subsample': [0.8, 1],
            'learning_rate': [0.05, 0.1]
   }


#Create model
lgbm_model = lgb.LGBMRegressor(random_state= 101, n_jobs = -1)

#Define RandomSearchCV with the custom scorer
lgbm_grid = GridSearchCV(lgbm_model, param_grid = param_grid_lgbm, scoring = rmse_scorer, 
                                  cv = KFold(n_splits = 5, shuffle = True, random_state = 101), verbose = 3)

#Perform grid search
lgbm_grid.fit(X_encoded_train, y_encoded_train)

#Get the best fitted model and measure training time
best_lgbm_model = lgbm_grid.best_estimator_
start_training_time_lgbm = time.time()
best_lgbm_model.fit(X_encoded_train, y_encoded_train)
training_time_lgbm = time.time() - start_training_time_lgbm

#Measure prediction time on test data
start_prediction_time_lgbm = time.time()
predictions_lgbm = best_lgbm_model.predict(X_encoded_test)
prediction_time_lgbm = time.time() - start_prediction_time_lgbm

# Evaluate the model on the test set
test_rmse_lgbm = np.sqrt(mean_squared_error(y_encoded_test, predictions_lgbm))

#Perform cross-validation with the best estimator and custom scorer
scores_lgbm = cross_val_score(best_lgbm_model, X_encoded_train, y_encoded_train, cv=5, scoring=rmse_scorer)

#Print the cross-validation scores
print("RMSE:", np.abs(scores_lgbm.mean()))
print("Test RMSE:", test_rmse_lgbm)

# Print the training results
print("Training time:", training_time_lgbm)
print("Prediction time:", prediction_time_lgbm)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=0.8; total time=   0.3s
[CV 2/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=0.8; total time=   0.4s
[CV 3/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=0.8; total time=   0.3s
[CV 4/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=0.8; total time=   0.4s
[CV 5/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=0.8; total time=   0.3s
[CV 1/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=1; total time=   0.4s
[CV 2/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=1; total time=   0.3s
[CV 3/5] END learning_rate=0.05, max_depth=1, n_estimators=50, num_leaves=31, subsample=1; total time=   0.4s
[CV 4/5] END learning_rate=0.05, max_depth=1, n_

[CV 4/5] END learning_rate=0.05, max_depth=3, n_estimators=100, num_leaves=63, subsample=0.8; total time=   0.7s
[CV 5/5] END learning_rate=0.05, max_depth=3, n_estimators=100, num_leaves=63, subsample=0.8; total time=   0.8s
[CV 1/5] END learning_rate=0.05, max_depth=3, n_estimators=100, num_leaves=63, subsample=1; total time=   0.8s
[CV 2/5] END learning_rate=0.05, max_depth=3, n_estimators=100, num_leaves=63, subsample=1; total time=   0.8s
[CV 3/5] END learning_rate=0.05, max_depth=3, n_estimators=100, num_leaves=63, subsample=1; total time=   0.7s
[CV 4/5] END learning_rate=0.05, max_depth=3, n_estimators=100, num_leaves=63, subsample=1; total time=   0.7s
[CV 5/5] END learning_rate=0.05, max_depth=3, n_estimators=100, num_leaves=63, subsample=1; total time=   0.8s
[CV 1/5] END learning_rate=0.05, max_depth=12, n_estimators=50, num_leaves=31, subsample=0.8; total time=   0.7s
[CV 2/5] END learning_rate=0.05, max_depth=12, n_estimators=50, num_leaves=31, subsample=0.8; total time= 

[CV 3/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=31, subsample=1; total time=   0.5s
[CV 4/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=31, subsample=1; total time=   0.5s
[CV 5/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=31, subsample=1; total time=   0.5s
[CV 1/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=63, subsample=0.8; total time=   0.5s
[CV 2/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=63, subsample=0.8; total time=   0.6s
[CV 3/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=63, subsample=0.8; total time=   0.5s
[CV 4/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=63, subsample=0.8; total time=   0.5s
[CV 5/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=63, subsample=0.8; total time=   0.6s
[CV 1/5] END learning_rate=0.1, max_depth=1, n_estimators=100, num_leaves=63, subsample=1; total time=   0.7s


[CV 2/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=0.8; total time=   1.2s
[CV 3/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=0.8; total time=   2.3s
[CV 4/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=0.8; total time=   1.1s
[CV 5/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=0.8; total time=   1.3s
[CV 1/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=1; total time=   1.3s
[CV 2/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=1; total time=   1.1s
[CV 3/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=1; total time=   1.4s
[CV 4/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=1; total time=   1.3s
[CV 5/5] END learning_rate=0.1, max_depth=12, n_estimators=100, num_leaves=31, subsample=1; total time= 

In [44]:
#CatBoost with hyperparameter tuning (optional)

#Define the parameter grid
param_grid_catboost = {
    'iterations': [100],
    'depth': [4, 6],
    'learning_rate': [0.05, 0.1],
    'subsample': [1.0],
    'rsm': [1.0]
}


#Create model
catboost_model = CatBoostRegressor(random_state= 101, cat_features = categorical_features, early_stopping_rounds=10)

#Define GridSearchCV with the custom scorer
catboost_grid = RandomizedSearchCV(catboost_model, param_distributions=param_grid_catboost, 
                                   n_iter=10, 
                                   scoring=rmse_scorer, 
                                   cv=KFold(n_splits=3, shuffle=True, random_state=101), 
                                   verbose=1, random_state=101)

# Perform grid search
catboost_grid.fit(X_original_train, y_original_train)

# Get the best fitted model and measure training time
best_catboost_model = catboost_grid.best_estimator_
start_training_time_catboost = time.time()
best_catboost_model.fit(X_original_train, y_original_train)
training_time_catboost = time.time() - start_training_time_catboost

# Measure prediction time
start_prediction_time_catboost = time.time()
predictions_catboost = best_catboost_model.predict(X_original_test)
prediction_time_catboost = time.time() - start_prediction_time_catboost

# Evaluate the model on the test set
test_rmse_catboost = np.sqrt(mean_squared_error(y_original_test, predictions_catboost))

# Perform cross-validation with the best estimator and custom scorer
scores_catboost = cross_val_score(best_catboost_model, X_original_train, y_original_train, cv=5, scoring=rmse_scorer)

#Print the cross-validation scores
print("RMSE:", np.abs(scores_catboost.mean()))
print("Test RMSE:", test_rmse_catboost)

# Print the training results
print("Training time:", training_time_catboost)
print("Prediction time:", prediction_time_catboost)

/opt/conda/envs/python3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 4 candidates, totalling 12 fits
0:	learn: 4907.5820528	total: 16ms	remaining: 1.59s
1:	learn: 4759.7916982	total: 28.8ms	remaining: 1.41s
2:	learn: 4622.4132854	total: 41.5ms	remaining: 1.34s
3:	learn: 4491.2129328	total: 54.3ms	remaining: 1.3s
4:	learn: 4373.1343741	total: 68.6ms	remaining: 1.3s
5:	learn: 4259.3892062	total: 82.6ms	remaining: 1.29s
6:	learn: 4152.3190100	total: 95.3ms	remaining: 1.27s
7:	learn: 4049.7767861	total: 108ms	remaining: 1.25s
8:	learn: 3954.9379005	total: 124ms	remaining: 1.25s
9:	learn: 3867.5158118	total: 137ms	remaining: 1.23s
10:	learn: 3783.8650423	total: 150ms	remaining: 1.21s
11:	learn: 3703.3993647	total: 164ms	remaining: 1.2s
12:	learn: 3630.4999047	total: 176ms	remaining: 1.18s
13:	learn: 3560.3648844	total: 189ms	remaining: 1.16s
14:	learn: 3495.0566435	total: 202ms	remaining: 1.14s
15:	learn: 3433.8583252	total: 215ms	remaining: 1.13s
16:	learn: 3376.9943681	total: 228ms	remaining: 1.11s
17:	learn: 3325.6039139	total:

62:	learn: 2411.1639144	total: 835ms	remaining: 490ms
63:	learn: 2405.0802187	total: 848ms	remaining: 477ms
64:	learn: 2398.6962158	total: 860ms	remaining: 463ms
65:	learn: 2393.0709719	total: 873ms	remaining: 450ms
66:	learn: 2388.3735911	total: 886ms	remaining: 436ms
67:	learn: 2383.3896233	total: 899ms	remaining: 423ms
68:	learn: 2378.1716205	total: 912ms	remaining: 410ms
69:	learn: 2373.8547921	total: 925ms	remaining: 396ms
70:	learn: 2368.9297576	total: 937ms	remaining: 383ms
71:	learn: 2364.2975249	total: 950ms	remaining: 369ms
72:	learn: 2359.7773957	total: 962ms	remaining: 356ms
73:	learn: 2355.6931610	total: 975ms	remaining: 342ms
74:	learn: 2352.2592611	total: 987ms	remaining: 329ms
75:	learn: 2349.0232886	total: 999ms	remaining: 315ms
76:	learn: 2346.0102439	total: 1.01s	remaining: 302ms
77:	learn: 2342.6338845	total: 1.02s	remaining: 289ms
78:	learn: 2339.5526731	total: 1.03s	remaining: 275ms
79:	learn: 2336.2667697	total: 1.05s	remaining: 262ms
80:	learn: 2333.0667671	tota

16:	learn: 2766.0744661	total: 233ms	remaining: 1.14s
17:	learn: 2718.9715562	total: 247ms	remaining: 1.12s
18:	learn: 2681.3021261	total: 259ms	remaining: 1.1s
19:	learn: 2647.3768399	total: 272ms	remaining: 1.09s
20:	learn: 2616.9029704	total: 284ms	remaining: 1.07s
21:	learn: 2590.4924094	total: 297ms	remaining: 1.05s
22:	learn: 2564.1912338	total: 310ms	remaining: 1.04s
23:	learn: 2536.0936582	total: 323ms	remaining: 1.02s
24:	learn: 2510.5555023	total: 337ms	remaining: 1.01s
25:	learn: 2487.0776771	total: 350ms	remaining: 995ms
26:	learn: 2467.4716936	total: 363ms	remaining: 980ms
27:	learn: 2449.5309411	total: 375ms	remaining: 965ms
28:	learn: 2435.3195261	total: 387ms	remaining: 949ms
29:	learn: 2420.9158735	total: 400ms	remaining: 933ms
30:	learn: 2406.0591476	total: 412ms	remaining: 917ms
31:	learn: 2393.6998861	total: 425ms	remaining: 903ms
32:	learn: 2382.6090094	total: 439ms	remaining: 891ms
33:	learn: 2371.0387860	total: 452ms	remaining: 877ms
34:	learn: 2360.9507296	total

76:	learn: 2211.1833623	total: 1.04s	remaining: 310ms
77:	learn: 2209.7796128	total: 1.05s	remaining: 296ms
78:	learn: 2208.5284294	total: 1.06s	remaining: 282ms
79:	learn: 2207.0729768	total: 1.07s	remaining: 268ms
80:	learn: 2205.6512200	total: 1.09s	remaining: 256ms
81:	learn: 2202.7522794	total: 1.1s	remaining: 242ms
82:	learn: 2200.3908200	total: 1.11s	remaining: 228ms
83:	learn: 2198.1322694	total: 1.13s	remaining: 215ms
84:	learn: 2196.7854540	total: 1.14s	remaining: 201ms
85:	learn: 2193.3521886	total: 1.16s	remaining: 188ms
86:	learn: 2191.0706610	total: 1.17s	remaining: 175ms
87:	learn: 2188.3238067	total: 1.18s	remaining: 161ms
88:	learn: 2187.2557424	total: 1.2s	remaining: 148ms
89:	learn: 2186.1468277	total: 1.21s	remaining: 135ms
90:	learn: 2184.6278622	total: 1.22s	remaining: 121ms
91:	learn: 2182.9948531	total: 1.24s	remaining: 108ms
92:	learn: 2180.3951564	total: 1.25s	remaining: 94ms
93:	learn: 2179.1051655	total: 1.26s	remaining: 80.5ms
94:	learn: 2176.6270013	total:

...

56:	learn: 2137.7527320	total: 1.55s	remaining: 1.17s
57:	learn: 2134.6991911	total: 1.57s	remaining: 1.14s
58:	learn: 2130.9979520	total: 1.6s	remaining: 1.11s
59:	learn: 2128.5914946	total: 1.63s	remaining: 1.08s
60:	learn: 2125.3706295	total: 1.65s	remaining: 1.05s
61:	learn: 2123.5202310	total: 1.68s	remaining: 1.03s
62:	learn: 2120.9565525	total: 1.7s	remaining: 1000ms
63:	learn: 2117.9998491	total: 1.73s	remaining: 972ms
64:	learn: 2114.8439198	total: 1.75s	remaining: 945ms
65:	learn: 2113.0136762	total: 1.78s	remaining: 917ms
66:	learn: 2110.5262336	total: 1.81s	remaining: 890ms
67:	learn: 2108.4272871	total: 1.84s	remaining: 864ms
68:	learn: 2106.4255944	total: 1.86s	remaining: 837ms
69:	learn: 2104.8452090	total: 1.89s	remaining: 810ms
70:	learn: 2102.9051988	total: 1.92s	remaining: 782ms
71:	learn: 2101.0073902	total: 1.94s	remaining: 755ms
72:	learn: 2099.0171377	total: 1.97s	remaining: 728ms
73:	learn: 2096.2243950	total: 1.99s	remaining: 700ms
74:	learn: 2094.2744504	total

9:	learn: 3081.3477690	total: 233ms	remaining: 2.1s
10:	learn: 2979.7731024	total: 257ms	remaining: 2.08s
11:	learn: 2894.1810804	total: 280ms	remaining: 2.05s
12:	learn: 2817.0925359	total: 306ms	remaining: 2.04s
13:	learn: 2749.4861711	total: 326ms	remaining: 2s
14:	learn: 2693.3754085	total: 347ms	remaining: 1.97s
15:	learn: 2645.1539579	total: 368ms	remaining: 1.93s
16:	learn: 2601.4331985	total: 390ms	remaining: 1.9s
17:	learn: 2564.9021285	total: 411ms	remaining: 1.87s
18:	learn: 2524.0251701	total: 431ms	remaining: 1.84s
19:	learn: 2487.6905798	total: 453ms	remaining: 1.81s
20:	learn: 2456.2022449	total: 475ms	remaining: 1.78s
21:	learn: 2431.0659388	total: 500ms	remaining: 1.77s
22:	learn: 2406.1711475	total: 521ms	remaining: 1.74s
23:	learn: 2385.0524833	total: 541ms	remaining: 1.71s
24:	learn: 2360.4763536	total: 563ms	remaining: 1.69s
25:	learn: 2338.2781285	total: 585ms	remaining: 1.66s
26:	learn: 2321.4720281	total: 606ms	remaining: 1.64s
27:	learn: 2305.4109740	total: 627

67:	learn: 2126.1923440	total: 1.49s	remaining: 703ms
68:	learn: 2120.4261958	total: 1.52s	remaining: 682ms
69:	learn: 2117.5091196	total: 1.54s	remaining: 660ms
70:	learn: 2114.4471396	total: 1.56s	remaining: 637ms
71:	learn: 2110.1720648	total: 1.58s	remaining: 615ms
72:	learn: 2107.4118270	total: 1.6s	remaining: 593ms
73:	learn: 2106.2355259	total: 1.62s	remaining: 570ms
74:	learn: 2102.5155252	total: 1.64s	remaining: 548ms
75:	learn: 2100.6744132	total: 1.67s	remaining: 526ms
76:	learn: 2097.1967476	total: 1.69s	remaining: 504ms
77:	learn: 2094.9825040	total: 1.71s	remaining: 483ms
78:	learn: 2092.2043347	total: 1.73s	remaining: 461ms
79:	learn: 2092.2040631	total: 1.74s	remaining: 435ms
80:	learn: 2091.4641918	total: 1.76s	remaining: 413ms
81:	learn: 2089.2622958	total: 1.78s	remaining: 391ms
82:	learn: 2087.9251083	total: 1.8s	remaining: 370ms
83:	learn: 2085.6330390	total: 1.82s	remaining: 348ms
84:	learn: 2083.6589978	total: 1.85s	remaining: 326ms
85:	learn: 2081.6791019	total:

29:	learn: 2289.8113769	total: 667ms	remaining: 1.55s
30:	learn: 2279.5774956	total: 688ms	remaining: 1.53s
31:	learn: 2270.1567316	total: 709ms	remaining: 1.51s
32:	learn: 2261.5724081	total: 731ms	remaining: 1.48s
33:	learn: 2254.8466357	total: 752ms	remaining: 1.46s
34:	learn: 2245.5396605	total: 774ms	remaining: 1.44s
35:	learn: 2239.2126304	total: 796ms	remaining: 1.41s
36:	learn: 2232.2552504	total: 818ms	remaining: 1.39s
37:	learn: 2224.7419006	total: 840ms	remaining: 1.37s
38:	learn: 2219.7020110	total: 861ms	remaining: 1.35s
39:	learn: 2215.3706166	total: 882ms	remaining: 1.32s
40:	learn: 2210.2032813	total: 904ms	remaining: 1.3s
41:	learn: 2206.2553258	total: 930ms	remaining: 1.28s
42:	learn: 2200.4619727	total: 953ms	remaining: 1.26s
43:	learn: 2196.1742852	total: 974ms	remaining: 1.24s
44:	learn: 2192.5005916	total: 996ms	remaining: 1.22s
45:	learn: 2187.5718918	total: 1.02s	remaining: 1.19s
46:	learn: 2183.7308807	total: 1.04s	remaining: 1.17s
47:	learn: 2180.5730552	total

86:	learn: 2069.7858183	total: 1.92s	remaining: 286ms
87:	learn: 2068.4307792	total: 1.94s	remaining: 264ms
88:	learn: 2066.2080501	total: 1.96s	remaining: 242ms
89:	learn: 2063.6899929	total: 1.98s	remaining: 220ms
90:	learn: 2062.3051012	total: 2s	remaining: 198ms
91:	learn: 2060.9421034	total: 2.02s	remaining: 176ms
92:	learn: 2059.5202035	total: 2.04s	remaining: 154ms
93:	learn: 2057.7142699	total: 2.06s	remaining: 132ms
94:	learn: 2056.6237363	total: 2.08s	remaining: 110ms
95:	learn: 2054.3646527	total: 2.1s	remaining: 87.7ms
96:	learn: 2051.6095463	total: 2.13s	remaining: 65.8ms
97:	learn: 2048.8862296	total: 2.15s	remaining: 43.9ms
98:	learn: 2046.2441855	total: 2.17s	remaining: 21.9ms
99:	learn: 2044.9602138	total: 2.19s	remaining: 0us
RMSE: 2071.405636547941
Test RMSE: 2064.30355961189
Training time: 2.9004037380218506
Prediction time: 0.018863201141357422
CPU times: user 37.2 s, sys: 248 ms, total: 37.4 s
Wall time: 39.2 s


<div class="alert alert-block alert-danger">
<b>Reviewer's comment V1</b> <a class="tocSkip"></a>

Well done! Everything is correct. You need just to fix two small issues:
1. The best model should be tested on the test data as usual
2. For each model you need to measure two separate times: training time and prediction time. To do it, you can use library `time`. But keep in mind, that grid search time is not the same as model traning time. To measure model training time you need to select the best model, retrain it on train data and measure this time.
    
</div>

<div class="alert alert-block alert-info">
<b>Student answer.</b> <a class="tocSkip"></a>

1. Updated code to check Test RMSE for all models not sure the best one<br>
2. I added code for each model on training time with help from Dot and a tutor in Discord
</div>

<div class="alert alert-block alert-success">
<b>Reviewer's comment V2</b> <a class="tocSkip"></a>

Now everything is correct. Good job!
    
</div>

## Model analysis

I was able to improve the models enough to have better RMSE and Test RMSE scores than the baseline linear regression model.

The most significant improvement was the CatBoost model w/ and RMSE of 2071 compared to 3370 baseline
and a Test RMSE of 2064 compared to 6281 baseline.

CatBoost also had the longest training time compared to the other models. It was about double that of Random Forest and LightGBM.

Baseline Linear Regression:<br>
RMSE: 3370.619291921021<br>
Test RMSE: 6281.211347513317<br>

Decision Tree:<br>
RMSE: 2516.4901436058767<br>
Test RMSE: 2495.5551406899185<br>
Training time: 0.23401117324829102<br>
Prediction time: 0.003443479537963867<br>

Random Forest:<br>
RMSE: 2389.4091770580676<br>
Test RMSE: 2366.2767131549053<br>
Training time: 1.4818060398101807<br>
Prediction time: 0.01737833023071289<br>

LightGBM:<br>
RMSE: 2143.8576132850567<br>
Test RMSE: 2121.5609175572217<br>
Training time: 1.5945460796356201<br>
Prediction time: 0.20604872703552246<br>

CatBoost:<br>
RMSE: 2071.405636547941<br>
Test RMSE: 2064.30355961189<br>
Training time: 2.9004037380218506<br>
Prediction time: 0.018863201141357422<br>

I had to decrease the parameter grid metrics a few times on a few of the models to get them to train in an acceptable timeframe.

# Checklist

Type 'x' to check. Then press Shift+Enter.

- [x]  Jupyter Notebook is open
- [x]  Code is error free
- [x]  The cells with the code have been arranged in order of execution
- [x]  The data has been downloaded and prepared
- [x]  The models have been trained
- [x]  The analysis of speed and quality of the models has been performed